# Load CSVs (one-to-many) to Azure Cognitive Search

In this Jupyter Notebook, we create and run steps to index a CSV file in which each row is an indivual and independent record/document. Each row then becomes searchable in Azure Cognitive Search. 
The reference documentation can be found at [Indexing blobs and files to produce multiple search documents](https://learn.microsoft.com/en-us/azure/search/search-howto-index-one-to-many-blobs).

By default, an indexer will treat the contents of a blob or file as a single search document. If you want a more granular representation in a search index, you can set parsingMode values to create multiple search documents from one blob or file.

We are going to be using the same private Blob Storage account but a different container that has abstracts of 90k Medical publications about COVID-19 published in 2020-2022. This file is a subset of a much bigger dataset (770k articles) called CORD19 [HERE](https://github.com/allenai/cord19)

In [1]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# Set the Data source connection string. This is the location of the CSV with the COVID articles on each line. 
# You can change it and use your own data
DATASOURCE_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=njonpdadls3;AccountKey=2MFKq8vzZVrxm4grSGo/cRbgkHqDuCFAoz0Yulx2D0wKBp879smmlrUZDcKeztL7M84pvQczHLWM+AStBLZ1SA==;EndpointSuffix=core.windows.net"
#DATASOURCE_SAS_TOKEN = "?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2024-10-02T01:02:07Z&st=2023-08-03T17:02:07Z&spr=https&sig=gLxStXFSY6X29OPpPDpBEhoQDdtJNDrMVExNYJ%2BhmBQ%3D"
DATASOURCE_SAS_TOKEN = "?sp=r&st=2023-08-29T23:09:13Z&se=2024-08-30T07:09:13Z&sv=2022-11-02&sr=c&sig=6peY%2BQUCV%2FfK5Rf6S86nt5chXpxnCJXkl2DVbaK0i2Q%3D"
#BLOB_CONTAINER_NAME = "arxivcs"
BLOB_CONTAINER_NAME = "customer360"
BLOB_FOLDER_NAME = "mapping"

In [2]:
# Define the names for the data source, index and indexer
datasource_name = "kyc-cogsrch-datasource-mappingcsv"
skillset_name = "kyc-cogsrch-skillset-mappingcsv"
index_name = "kyc-cogsrch-index-mcsv"
indexer_name = "kyc-cogsrch-indexer-mcsv"

In [3]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Create Data Source (Blob container with the Litcovid CSV data file)

In [4]:
# Create a data source

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities of one-to-many.",
    "type": "azureblob",
    "credentials": {
        "connectionString": DATASOURCE_CONNECTION_STRING
    },
    "container": {
        "name": BLOB_CONTAINER_NAME,
        "query" : BLOB_FOLDER_NAME
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


## Inspect CSV file so we can understand the column types before creating the Index

In our private dataset we have place a smaller version of the original the metadata.csv file in the cord19 dataset. 
Let's see what the file looks like:

In [5]:
#Download the csv files to disk and inspect using pandas
import pandas as pd
remote_file_path = "https://njonpdadls3.blob.core.windows.net/customer360/mapping/Mapping.csv"

In [6]:
metadata = pd.read_csv(remote_file_path + DATASOURCE_SAS_TOKEN)
print("No. of lines:",metadata.shape[0])

#simple_schema = ['TPID', 'CustomerName', 'AECovered']
#simple_schema = ['TPID', 'CustomerName', 'AECovered', 'SubSegment', 'Subsidiary', 'SalesTerritory', 'Vertical', 'VerticalCategory', 'AE', 'AzureMotion', 'AzureCoreSpecialist', 'AppInno', 'AzureD&AISpecialist', 'BizAppsSpecialist', 'LowCodeMotion', 'LowCodeSpecialist', 'ModernWorkMotion', 'ModernWorkSpecialist', 'SecuritySpecialist', 'SurfaceSpecialist' ]
simple_schema = ['TPID', 'CustomerName', 'AECovered', 'Vertical', 'VerticalCategory', 'AE', 'AzureMotion', 'AzureCoreSpecialist', 'AppInno', 'AzureDAISpecialist']

# def make_clickable(address):
#     '''Make the url clickable'''
#     return '<a href="{0}">{0}</a>'.format(address)

# def preview(text):
#     '''Show only a preview of the text data.'''
#     return text[:30] + '...'

#format_ = {'title': preview, 'abstract': preview, 'authors': preview, 'url': make_clickable}

#metadata[simple_schema].head().style.format(format_)
metadata[simple_schema].head()

No. of lines: 339


,TPID,CustomerName,AECovered,Vertical,VerticalCategory,AE,AzureMotion,AzureCoreSpecialist,AppInno,AzureDAISpecialist
0,993255,MISSION FEDERAL CREDIT UNION,SMC AE Covered,Banking,Credit Unions,ANSCOBEE,SMC Acq Azure,ZAJOHNS,NaN,NaN
1,991225,Kansas City Life,SMC Non AE Covered,Insurance,Direct Life Insurance Carriers,NaN,SMC Acq Azure,ZAJOHNS,NaN,NaN
2,9900283,Inter National Bank,SMC AE Covered,Banking,Commercial Banking,PASTURGI,SMC Grow Azure,"MICHELLEKING, KHANRAHAN",ERHALDAN,MWILLEY
3,989570,HELIX ELECTRIC INC,SMC AE Covered,Real Estate,Construction of Infrastructure,ANSCOBEE,SMC Grow Azure,"MICHELLEKING, KEVCARTER",ERHALDAN,BRIANJISHI
4,9689842,Futuredontics,SMC Non AE Covered,IT Services & Business Advisory,Managed IT Services,NaN,SMC Grow Azure,"MICHELLEKING, KHANRAHAN",ERHALDAN,MWILLEY


## Create Skillset - Text Splitter, Language Detection
We will use cognitive services enrichment for spliting the text of each content field into chunks (pages) and for language detection. We should always split the text since we don't know how big the content of each row might be.

In [7]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description": "Splits Text and detect language",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
            "description": "If you have multilingual content, adding a language code is useful for filtering",
            "context": "/document",
            "inputs": [
                {
                  "name": "text",
                  "source": "/document/abstract"
                }
            ],
            "outputs": [
                {
                  "name": "languageCode",
                  "targetName": "language"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",
            "maximumPageLength": 5000, # 5000 is default
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/abstract"
                },
                {
                    "name": "languageCode",
                    "source": "/document/language"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "pages"
                }
            ]
        }
    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


## Create the Index
In Azure Cognitive Search, both blob indexers and file indexers support a delimitedText parsing mode for CSV files that treats each line in the CSV as a separate search document.

### **Important**:
As you can see below and from the prior Notebook, there are 7 mandatory fields in the schema: `id, title, content, chunks, language, name, location`. These fields must exist in any index that you create regardles of the datasource. Any additional fields are good to add so the engine can search relevant documents, however the mandatory fields must exist for all the code downstream work with no issues.

In [8]:
index_payload = {
    "name": index_name,  
    "fields": [
        {"name": "TPID", "type": "Edm.String", "key": "true", "searchable": "false", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "CustomerName", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "AECovered", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "Vertical","type": "Collection(Edm.String)", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "VerticalCategory", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "true", "filterable": "true", "facetable": "true"},
        {"name": "AE", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "AzureMotion", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "AzureCoreSpecialist", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "AppInno", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "AzureDAISpecialist", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "authors", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "metadata_storage_name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "metadata_storage_path", "type":"Edm.String", "searchable": "false", "retrievable": "true", "filterable": "false", "sortable": "false"},
        {"name": "metadata_storage_last_modified", "type":"Edm.DateTimeOffset", "searchable": "false", "retrievable": "false", "filterable": "false", "sortable": "false"}
    ],
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": 
                        {
                            "fieldName": "CustomerName"
                        },
                    "prioritizedContentFields": [
                        { 
                            "fieldName":"Vertical" ,
                            "fieldName":"AECovered" ,
                            "fieldName":"AzureMotion" ,
                            "fieldName":"AzureCoreSpecialist" ,
                            "fieldName":"AppInno" ,
                            "fieldName":"AzureDAISpecialist" ,                            

                        }
                    ]
                }
            }
        ]
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


In [9]:
print(r.text)

## Create and Run the Indexer - (runs the pipeline)
To create one-to-many indexers with CSV blobs, create or update an indexer definition with the delimitedText parsing mode

In [10]:
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "schedule" : { "interval" : "PT2H"},
    "fieldMappings": [
        {
          "sourceFieldName" : "TPID",
          "targetFieldName" : "TPID"
        },
        {
          "sourceFieldName" : "CustomerName",
          "targetFieldName" : "CustomerName"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "url",
          "targetFieldName" : "location"
        }
    ],
    "outputFieldMappings":
    [
        {
            "sourceFieldName": "/document/language",
            "targetFieldName": "language"
        },
        {
            "sourceFieldName": "/document/pages/*",
            "targetFieldName": "chunks"
        }
    ],
    "parameters" : { 
        "configuration" : { 
            "dataToExtract": "contentAndMetadata",
            "parsingMode" : "delimitedText", 
            "firstLineContainsHeaders" : True,
            "delimitedTextDelimiter": ","
        } 
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

400
False


In [11]:
print(r.text)

{"error":{"code":"","message":"Output field mapping specifies target field 'language' that doesn't exist in the index"}}


In [49]:
# Optionally, get indexer status to confirm that it's running
r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                 "/status", headers=headers, params=params)
# pprint(json.dumps(r.json(), indent=1))
print(r.status_code)
print("Status:",r.json().get('lastResult').get('status'))
print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
print(r.ok)

200
Status: success
Items Processed: 90000
True


**When the indexer finishes running we will have all 90,000 rows indexed properly as separate documents in our Search Engine!.**

# Reference

- https://learn.microsoft.com/en-us/azure/search/search-howto-index-csv-blobs
- https://learn.microsoft.com/en-us/azure/search/knowledge-store-create-rest



# NEXT
Now that we have two separete indexes loaded with two different types of information, In the next notebook 3, we will do a Multi-Index query, sort the results based on the reranker semantic score of Azure Search, and then use OpenAI to understand this results and give the best answer possible